# 1. SETTINGS

In [1]:
# libraries
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

In [2]:
# pandas options
pd.set_option("display.max_columns", None)

In [3]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
# garbage collection
import gc
gc.enable()

# 2. FUNCTIONS

In [5]:
##### FUNCTION FOR COUNTING MISSINGS
def count_missings(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum() / data.isnull().count() * 100).sort_values(ascending = False)
    table = pd.concat([total, percent], axis = 1, keys = ["Total", "Percent"])
    table = table[table["Total"] > 0]
    return table

In [6]:
##### FUNCTION FOR CREATING LOGARITHMS
def convert_days(data, features, t = 12, rounding = True, replace = False):
    for var in features:
        if replace == True:
            if rounding == True:
                data[var] = round(-data[var]/t)
            else:
                data[var] = -data[var]/t
            data[var][data[var] < 0] = None
        else:
            if rounding == True:
                data["CONVETRTED_" + str(var)] = round(-data[var]/t)
            else:
                data["CONVETRTED_" + str(var)] = -data[var]/t
            data["CONVETRTED_" + str(var)][data["CONVETRTED_" + str(var)] < 0] = None
    return data

In [7]:
##### FUNCTION FOR CREATING LOGARITHMS
def create_logs(data, features, replace = False):
    for var in features:
        if replace == True:
            data[var] = np.log(data[var].abs() + 1)
        else:
            data["LOG_" + str(var)] = np.log(data[var].abs() + 1)      
    return data

In [8]:
##### FUNCTION FOR CREATING FLAGS FOR MISSINGS
def create_null_flags(data, features = None):
    if features == None:
        features = data.columns
    for var in features:
        num_null = data[var].isnull() + 0
        if num_null.sum() > 0:
            data["ISNULL_" + str(var)] = num_null
    return data

In [9]:
##### FUNCTION FOR TREATING FACTORS
def treat_factors(data, method = "label"):
    
    # label encoding
    if method == "label":
        factors = [f for f in data.columns if data[f].dtype == "object"]
        for var in factors:
            data[var], _ = pd.factorize(data[var])
        
    # dummy encoding
    if method == "dummy":
        data = pd.get_dummies(data, drop_first = True)
    
    # dataset
    return data

In [10]:
##### FUNCTION FOR COMPUTING ACCEPT/REJECT RATIOS
def compute_accept_reject_ratio(data, lags = [1, 3, 5]):
    
    # preparations
    dec_prev = data[["SK_ID_CURR", "SK_ID_PREV", "DAYS_DECISION", "NAME_CONTRACT_STATUS"]]
    dec_prev["DAYS_DECISION"] = -dec_prev["DAYS_DECISION"]
    dec_prev = dec_prev.sort_values(by = ["SK_ID_CURR", "DAYS_DECISION"])
    dec_prev = pd.get_dummies(dec_prev)
     
    # compuatation
    for t in lags:
        
        # acceptance ratios
        tmp = dec_prev[["SK_ID_CURR", "NAME_CONTRACT_STATUS_Approved"]].groupby(["SK_ID_CURR"]).head(1)
        tmp = tmp.groupby(["SK_ID_CURR"], as_index = False).mean()
        tmp.columns = ["SK_ID_CURR", "APPROVE_RATIO_" + str(t)]
        data = data.merge(tmp, how = "left", on = "SK_ID_CURR")
        
        # rejection ratios
        tmp = dec_prev[["SK_ID_CURR", "NAME_CONTRACT_STATUS_Refused"]].groupby(["SK_ID_CURR"]).head(1)
        tmp = tmp.groupby(["SK_ID_CURR"], as_index = False).mean()
        tmp.columns = ["SK_ID_CURR", "REJECT_RATIO_" + str(t)]
        data = data.merge(tmp, how = "left", on = "SK_ID_CURR")
        
    # dataset
    return data

In [11]:
##### FUNCTION FOR AGGREGATING FEATURES
def aggregate_data(data, id_var, label = None, stats = ["mean", "std", "min", "max"]):
    
    
    ### SEPARATE NUMERIC AND FACTORS
  
    # display info
    print("- Preparing the dataset...")
      
    # find factors
    data_factors = [f for f in data.columns if data[f].dtype == "object"]
    
    # partition subsets
    num_data = data[list(set(data.columns) - set(data_factors))]
    fac_data = data[[id_var] + data_factors]
    
    # display info
    num_facs = fac_data.shape[1] - 1
    num_nums = num_data.shape[1] - 1
    print("- Extracted %.0f factors and %.0f numerics..." % (num_facs, num_nums))


    ##### AGGREGATION
 
    # aggregate numerics
    if (num_nums > 0):
        print("- Aggregating numeric features...")
        num_data = num_data.groupby(id_var).agg(stats)
        num_data.columns = ["_".join(col).strip() for col in num_data.columns.values]
        num_data = num_data.sort_index()

    # aggregate factors
    if (num_facs > 0):
        print("- Aggregating factor features...")
        fac_data = fac_data.groupby(id_var).agg([("mode",    lambda x: scipy.stats.mode(x)[0][0]),
                                                 ("nunique", lambda x: x.nunique())])
        fac_data.columns = ["_".join(col).strip() for col in fac_data.columns.values]
        fac_data = fac_data.sort_index()


    ##### MERGER

    # merge numerics and factors
    if ((num_facs > 0) & (num_nums > 0)):
        agg_data = pd.concat([num_data, fac_data], axis = 1)
    
    # use factors only
    if ((num_facs > 0) & (num_nums == 0)):
        agg_data = fac_data
        
    # use numerics only
    if ((num_facs == 0) & (num_nums > 0)):
        agg_data = num_data
        

    ##### LAST STEPS

    # update labels
    if label != None:
        agg_data.columns = [label + "_" + str(col) for col in agg_data.columns]

    # display info
    print("- Final dimensions:", agg_data.shape)
    
    # return dataset
    return agg_data

# 3. DATA CHECKS

In [12]:
# import data
train = pd.read_csv("../data/raw/application_train.csv")
test  = pd.read_csv("../data/raw/application_test.csv")
buro  = pd.read_csv("../data/raw/bureau.csv")
bbal  = pd.read_csv("../data/raw/bureau_balance.csv")
prev  = pd.read_csv("../data/raw/previous_application.csv")
card  = pd.read_csv("../data/raw/credit_card_balance.csv")
poca  = pd.read_csv("../data/raw/POS_CASH_balance.csv")
inst  = pd.read_csv("../data/raw/installments_payments.csv")

In [13]:
# check dimensions
print("Application:", train.shape, test.shape)
print("Buro:", buro.shape)
print("Bbal:", bbal.shape)
print("Prev:", prev.shape)
print("Card:", card.shape)
print("Poca:", poca.shape)
print("Inst:", inst.shape)

Application: (307511, 122) (48744, 121)
Buro: (1716428, 17)
Bbal: (27299925, 3)
Prev: (1670214, 37)
Card: (3840312, 23)
Poca: (10001358, 8)
Inst: (13605401, 8)


In [14]:
### CHECK LOAN ID DISTRIBUTION

# check unique IDs
print("IDs in TRAIN:", train.SK_ID_CURR.nunique())
print("IDs in TEST:",  test.SK_ID_CURR.nunique())
print("IDs in BURO:",  buro.SK_ID_CURR.nunique())
print("IDs in PREV:",  prev.SK_ID_CURR.nunique())
print("IDs in CARD:",  card.SK_ID_CURR.nunique())
print("IDs in POCA:",  poca.SK_ID_CURR.nunique())
print("IDs in INST:",  inst.SK_ID_CURR.nunique())
print("")

# check current loan differences
print("IDs in TRAIN but not in BURO:", 
      len(list(set(train.SK_ID_CURR.unique()) - set(buro.SK_ID_CURR.unique()))))
print("IDs in TRAIN but not in PREV:", 
      len(list(set(train.SK_ID_CURR.unique()) - set(prev.SK_ID_CURR.unique()))))
print("IDs in TRAIN but not in CARD:", 
      len(list(set(train.SK_ID_CURR.unique()) - set(card.SK_ID_CURR.unique()))))
print("IDs in TRAIN but not in POCA:", 
      len(list(set(train.SK_ID_CURR.unique()) - set(poca.SK_ID_CURR.unique()))))
print("IDs in TRAIN but not in INST:", 
      len(list(set(train.SK_ID_CURR.unique()) - set(poca.SK_ID_CURR.unique()))))
print("")

# check current loan differences
print("IDs in TEST but not in BURO:", 
      len(list(set(test.SK_ID_CURR.unique()) - set(buro.SK_ID_CURR.unique()))))
print("IDs in TEST but not in PREV:", 
      len(list(set(test.SK_ID_CURR.unique()) - set(prev.SK_ID_CURR.unique()))))
print("IDs in TEST but not in CARD:", 
      len(list(set(test.SK_ID_CURR.unique()) - set(card.SK_ID_CURR.unique()))))
print("IDs in TEST but not in POCA:", 
      len(list(set(test.SK_ID_CURR.unique()) - set(poca.SK_ID_CURR.unique()))))
print("IDs in TEST but not in INST:", 
      len(list(set(test.SK_ID_CURR.unique()) - set(poca.SK_ID_CURR.unique()))))
print("")

print("IDs in TRAIN and BURO:", 
       len(set.intersection(set(train.SK_ID_CURR.unique()), set(buro.SK_ID_CURR.unique()))))
print("IDs in TRAIN and PREV:", 
       len(set.intersection(set(train.SK_ID_CURR.unique()), set(prev.SK_ID_CURR.unique()))))
print("IDs in TRAIN and CARD:", 
       len(set.intersection(set(train.SK_ID_CURR.unique()), set(card.SK_ID_CURR.unique())))) 
print("IDs in TRAIN and POCA:", 
       len(set.intersection(set(train.SK_ID_CURR.unique()), set(poca.SK_ID_CURR.unique())))) 
print("IDs in TRAIN and INST:", 
       len(set.intersection(set(train.SK_ID_CURR.unique()), set(inst.SK_ID_CURR.unique())))) 
print("IDs in TRAIN and BURO and PREV:", 
       len(set.intersection(set(train.SK_ID_CURR.unique()), set(buro.SK_ID_CURR.unique()), 
                            set(prev.SK_ID_CURR.unique()))))
print("IDs in TRAIN and PREV and POCA and INST:", 
       len(set.intersection(set(train.SK_ID_CURR.unique()), set(prev.SK_ID_CURR.unique()), 
                            set(poca.SK_ID_CURR.unique()), set(inst.SK_ID_CURR.unique()))))
print("")

print("IDs in TEST and BURO:", 
       len(set.intersection(set(test.SK_ID_CURR.unique()), set(buro.SK_ID_CURR.unique()))))
print("IDs in TEST and PREV:", 
       len(set.intersection(set(test.SK_ID_CURR.unique()), set(prev.SK_ID_CURR.unique()))))
print("IDs in TEST and CARD:", 
       len(set.intersection(set(test.SK_ID_CURR.unique()), set(card.SK_ID_CURR.unique())))) 
print("IDs in TEST and POCA:", 
       len(set.intersection(set(test.SK_ID_CURR.unique()), set(poca.SK_ID_CURR.unique())))) 
print("IDs in TEST and INST:", 
       len(set.intersection(set(test.SK_ID_CURR.unique()), set(inst.SK_ID_CURR.unique())))) 
print("IDs in TEST and BURO and PREV:", 
       len(set.intersection(set(test.SK_ID_CURR.unique()), set(buro.SK_ID_CURR.unique()), 
                            set(prev.SK_ID_CURR.unique()))))
print("IDs in TEST and PREV and POCA and INST:", 
       len(set.intersection(set(test.SK_ID_CURR.unique()), set(prev.SK_ID_CURR.unique()), 
                            set(poca.SK_ID_CURR.unique()), set(inst.SK_ID_CURR.unique()))))

IDs in TRAIN: 307511
IDs in TEST: 48744
IDs in BURO: 305811
IDs in PREV: 338857
IDs in CARD: 103558
IDs in POCA: 337252
IDs in INST: 339587

IDs in TRAIN but not in BURO: 44020
IDs in TRAIN but not in PREV: 16454
IDs in TRAIN but not in CARD: 220606
IDs in TRAIN but not in POCA: 18067
IDs in TRAIN but not in INST: 18067

IDs in TEST but not in BURO: 6424
IDs in TEST but not in PREV: 944
IDs in TEST but not in CARD: 32091
IDs in TEST but not in POCA: 936
IDs in TEST but not in INST: 936

IDs in TRAIN and BURO: 263491
IDs in TRAIN and PREV: 291057
IDs in TRAIN and CARD: 86905
IDs in TRAIN and POCA: 289444
IDs in TRAIN and INST: 291643
IDs in TRAIN and BURO and PREV: 249507
IDs in TRAIN and PREV and POCA and INST: 288028

IDs in TEST and BURO: 42320
IDs in TEST and PREV: 47800
IDs in TEST and CARD: 16653
IDs in TEST and POCA: 47808
IDs in TEST and INST: 47944
IDs in TEST and BURO and PREV: 41584
IDs in TEST and PREV and POCA and INST: 47537


# 4. PREPROCESSING

## 4.1. APPLICATION DATA

In [12]:
# import data
train = pd.read_csv("../data/raw/application_train.csv")
test  = pd.read_csv("../data/raw/application_test.csv")
print("Dimensions:", train.shape, test.shape)

Dimensions: (307511, 122) (48744, 121)


In [13]:
# extract target
y = train[["SK_ID_CURR", "TARGET"]]
del train["TARGET"]

In [14]:
# concatenate application data
appl = pd.concat([train, test])
del train, test

In [15]:
# check data
appl.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [16]:
# count missings
nas = count_missings(appl)
nas

,Total,Percent
COMMONAREA_MEDI,248360,69.714109
COMMONAREA_AVG,248360,69.714109
COMMONAREA_MODE,248360,69.714109
NONLIVINGAPARTMENTS_MODE,246861,69.293343
NONLIVINGAPARTMENTS_MEDI,246861,69.293343
NONLIVINGAPARTMENTS_AVG,246861,69.293343
FONDKAPREMONT_MODE,243092,68.235393
LIVINGAPARTMENTS_MEDI,242979,68.203674
LIVINGAPARTMENTS_MODE,242979,68.203674
LIVINGAPARTMENTS_AVG,242979,68.203674


In [17]:
# print factor levels
facs =  [f for f in appl.columns if appl[f].dtype == "object"]
for var in facs:
    print("----------")
    print(appl[var].value_counts())
    print(appl[var].isnull().sum(), " null values")
    print("")

----------
Cash loans         326537
Revolving loans     29718
Name: NAME_CONTRACT_TYPE, dtype: int64
0  null values

----------
F      235126
M      121125
XNA         4
Name: CODE_GENDER, dtype: int64
0  null values

----------
N    235235
Y    121020
Name: FLAG_OWN_CAR, dtype: int64
0  null values

----------
Y    246970
N    109285
Name: FLAG_OWN_REALTY, dtype: int64
0  null values

----------
Unaccompanied      288253
Family              46030
Spouse, partner     12818
Children             3675
Other_B              1981
Other_A               975
Group of people       320
Name: NAME_TYPE_SUITE, dtype: int64
2203  null values

----------
Working                 183307
Commercial associate     83019
Pensioner                64635
State servant            25235
Unemployed                  23
Student                     20
Businessman                 11
Maternity leave              5
Name: NAME_INCOME_TYPE, dtype: int64
0  null values

----------
Secondary / secondary special    252379

In [18]:
##### FEATURE ENGINEERING

# income ratios
appl["CREDIT_BY_INCOME"]      = appl["AMT_CREDIT"]      / appl["AMT_INCOME_TOTAL"]
appl["ANNUITY_BY_INCOME"]     = appl["AMT_ANNUITY"]     / appl["AMT_INCOME_TOTAL"]
appl["GOODS_PRICE_BY_INCOME"] = appl["AMT_GOODS_PRICE"] / appl["AMT_INCOME_TOTAL"]

# career ratio
appl["PERCENT_WORKED"] = appl["DAYS_EMPLOYED"] / appl["DAYS_BIRTH"]
appl["PERCENT_WORKED"][appl["PERCENT_WORKED"] < 0] = None

# number of adults
appl["CNT_ADULTS"] = appl["CNT_FAM_MEMBERS"] - appl["CNT_CHILDREN"]

# external sources
#appl["EXT_SOURCE_MEAN"] = appl[["EXT_SOURCE_1", "EXT_SOURCE_1", "EXT_SOURCE_3"]].mean(axis = 1)
#appl["EXT_SOURCE_SD"]   = appl[["EXT_SOURCE_1", "EXT_SOURCE_1", "EXT_SOURCE_3"]].std(axis = 1)
appl["NUM_EXT_SOURCES"] = 3 - (appl["EXT_SOURCE_1"].isnull().astype(int) +
                               appl["EXT_SOURCE_2"].isnull().astype(int) +
                               appl["EXT_SOURCE_3"].isnull().astype(int))

# number of documents
doc_vars = ["FLAG_DOCUMENT_2",  "FLAG_DOCUMENT_3",  "FLAG_DOCUMENT_4",  "FLAG_DOCUMENT_5",  "FLAG_DOCUMENT_6",
            "FLAG_DOCUMENT_7",  "FLAG_DOCUMENT_8",  "FLAG_DOCUMENT_9",  "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11",
            "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16",
            "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19", "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21"]
appl["NUM_DOCUMENTS"] = appl[doc_vars].sum(axis = 1)

# application date
appl["DAY_APPR_PROCESS_START"] = "Working day"
appl["DAY_APPR_PROCESS_START"][(appl["WEEKDAY_APPR_PROCESS_START"] == "SATURDAY") |
                               (appl["WEEKDAY_APPR_PROCESS_START"] == "SUNDAY")] = "Weekend"

# logarithms
log_vars = ["AMT_CREDIT", "AMT_INCOME_TOTAL", "AMT_GOODS_PRICE", "AMT_ANNUITY"]
appl = create_logs(appl, log_vars, replace = True)

# convert days
day_vars = ["DAYS_BIRTH", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "DAYS_EMPLOYED", "DAYS_LAST_PHONE_CHANGE"]
appl = convert_days(appl, day_vars, t = 30, rounding = True, replace = True)



##### FEATURE REMOVAL
drops = ['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 
         'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
         'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI',
         'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE','ELEVATORS_MODE', 'ENTRANCES_MODE', 
         'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 
         'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'TOTALAREA_MODE',  'YEARS_BEGINEXPLUATATION_MODE']
appl = appl.drop(columns = drops)



##### ADDING NULL FLAGS

# all null flags
#appl = create_null_flags(appl)

# buro enquiry flag
buro_vars = ["AMT_REQ_CREDIT_BUREAU_YEAR", "AMT_REQ_CREDIT_BUREAU_QRT", 
             "AMT_REQ_CREDIT_BUREAU_MON",  "AMT_REQ_CREDIT_BUREAU_WEEK", 
             "AMT_REQ_CREDIT_BUREAU_DAY",  "AMT_REQ_CREDIT_BUREAU_HOUR"]
appl["ISNULL_BURO_ENQUIRIES"] = (appl[buro_vars].isnull().sum(axis = 1) == 0).astype(int)

# social circle flag
social_vars = ["OBS_30_CNT_SOCIAL_CIRCLE", "DEF_30_CNT_SOCIAL_CIRCLE", "OBS_60_CNT_SOCIAL_CIRCLE", 
               "DEF_60_CNT_SOCIAL_CIRCLE", "AMT_GOODS_PRICE"]
appl["ISNULL_SOCIAL_CIRCLE"] = (appl[social_vars].isnull().sum(axis = 1) == 0).astype(int)



##### TREAT FACTORS

# merge some levels
#appl["CODE_GENDER"][appl["CODE_GENDER"] == "XNA"] = "F"
#appl["NAME_TYPE_SUITE"][(appl["NAME_TYPE_SUITE"] == "Other_B") | 
#                        (appl["NAME_TYPE_SUITE"] == "Other_A") |
#                        (appl["NAME_TYPE_SUITE"] == None)] = "Other"
#appl["NAME_INCOME_TYPE"][appl["NAME_INCOME_TYPE"] == "Businessman"] = "Working"
#appl["NAME_INCOME_TYPE"][(appl["NAME_INCOME_TYPE"] == "Student") |
#                         (appl["NAME_INCOME_TYPE"] == "Maternity leave")] = "Unemployed"
#appl["NAME_FAMILY_STATUS"][appl["NAME_FAMILY_STATUS"] == None] = "Unknown"
#appl["OCCUPATION_TYPE"][appl["OCCUPATION_TYPE"] == "IT staff"] = "High skill tech staff"
#appl["OCCUPATION_TYPE"][appl["OCCUPATION_TYPE"] == "HR staff"] = "Secretaries"
#appl["OCCUPATION_TYPE"][appl["NAME_INCOME_TYPE"] == "Unemployed"] = "No occupation"
#appl["ORGANIZATION_TYPE"][(appl["ORGANIZATION_TYPE"] == "XNA")] = "Other"
#appl["ORGANIZATION_TYPE"][(appl["ORGANIZATION_TYPE"] == "Business Entity Type 1") |
#                          (appl["ORGANIZATION_TYPE"] == "Business Entity Type 2") |
#                          (appl["ORGANIZATION_TYPE"] == "Business Entity Type 3")] = "Business Entity"
#appl["ORGANIZATION_TYPE"][(appl["ORGANIZATION_TYPE"] == "Transport: type 1") |
#                          (appl["ORGANIZATION_TYPE"] == "Transport: type 2") |
#                          (appl["ORGANIZATION_TYPE"] == "Transport: type 3") |
#                          (appl["ORGANIZATION_TYPE"] == "Transport: type 4")] = "Transport"
#appl["ORGANIZATION_TYPE"][(appl["ORGANIZATION_TYPE"] == "Industry: type 1") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 2") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 3") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 4") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 5") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 6") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 7") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 8") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 9") |
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 10")|
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 11")|
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 12")|
#                          (appl["ORGANIZATION_TYPE"] == "Industry: type 13")] = "Industry"
#appl["ORGANIZATION_TYPE"][(appl["ORGANIZATION_TYPE"] == "Trade: type 1") |
#                          (appl["ORGANIZATION_TYPE"] == "Trade: type 2") |
#                          (appl["ORGANIZATION_TYPE"] == "Trade: type 3") |
#                          (appl["ORGANIZATION_TYPE"] == "Trade: type 4") |
#                          (appl["ORGANIZATION_TYPE"] == "Trade: type 5") |
#                          (appl["ORGANIZATION_TYPE"] == "Trade: type 6") |
#                          (appl["ORGANIZATION_TYPE"] == "Trade: type 7")] = "Trade"
#appl["FONDKAPREMONT_MODE"][(appl["FONDKAPREMONT_MODE"] == None)] = "not specified"

# encoding
appl = treat_factors(appl, method = "dummy")

In [19]:
# rename features
appl = appl.reindex_axis(["SK_ID_CURR"] + list([col for col in appl.columns if col not in "SK_ID_CURR"]), axis = 1)
appl.columns = ["SK_ID_CURR"] + ["app_" + str(col) for col in appl.columns if col not in "SK_ID_CURR"]

In [20]:
# check data
appl.head()

,SK_ID_CURR,app_CNT_CHILDREN,app_AMT_INCOME_TOTAL,app_AMT_CREDIT,app_AMT_ANNUITY,app_AMT_GOODS_PRICE,app_REGION_POPULATION_RELATIVE,app_DAYS_BIRTH,app_DAYS_EMPLOYED,app_DAYS_REGISTRATION,app_DAYS_ID_PUBLISH,app_OWN_CAR_AGE,app_FLAG_MOBIL,app_FLAG_EMP_PHONE,app_FLAG_WORK_PHONE,app_FLAG_CONT_MOBILE,app_FLAG_PHONE,app_FLAG_EMAIL,app_CNT_FAM_MEMBERS,app_REGION_RATING_CLIENT,app_REGION_RATING_CLIENT_W_CITY,app_HOUR_APPR_PROCESS_START,app_REG_REGION_NOT_LIVE_REGION,app_REG_REGION_NOT_WORK_REGION,app_LIVE_REGION_NOT_WORK_REGION,app_REG_CITY_NOT_LIVE_CITY,app_REG_CITY_NOT_WORK_CITY,app_LIVE_CITY_NOT_WORK_CITY,app_EXT_SOURCE_1,app_EXT_SOURCE_2,app_EXT_SOURCE_3,app_APARTMENTS_AVG,app_BASEMENTAREA_AVG,app_YEARS_BEGINEXPLUATATION_AVG,app_YEARS_BUILD_AVG,app_COMMONAREA_AVG,app_ELEVATORS_AVG,app_ENTRANCES_AVG,app_FLOORSMAX_AVG,app_FLOORSMIN_AVG,app_LANDAREA_AVG,app_LIVINGAPARTMENTS_AVG,app_LIVINGAREA_AVG,app_NONLIVINGAPARTMENTS_AVG,app_NONLIVINGAREA_AVG,app_YEARS_BUILD_MODE,app_OBS_30_CNT_SOCIAL_CIRCLE,app_DEF_30_CNT_SOCIAL_CIRCLE,app_OBS_60_CNT_SOCIAL_CIRCLE,app_DEF_60_CNT_SOCIAL_CIRCLE,app_DAYS_LAST_PHONE_CHANGE,app_FLAG_DOCUMENT_2,app_FLAG_DOCUMENT_3,app_FLAG_DOCUMENT_4,app_FLAG_DOCUMENT_5,app_FLAG_DOCUMENT_6,app_FLAG_DOCUMENT_7,app_FLAG_DOCUMENT_8,app_FLAG_DOCUMENT_9,app_FLAG_DOCUMENT_10,app_FLAG_DOCUMENT_11,app_FLAG_DOCUMENT_12,app_FLAG_DOCUMENT_13,app_FLAG_DOCUMENT_14,app_FLAG_DOCUMENT_15,app_FLAG_DOCUMENT_16,app_FLAG_DOCUMENT_17,app_FLAG_DOCUMENT_18,app_FLAG_DOCUMENT_19,app_FLAG_DOCUMENT_20,app_FLAG_DOCUMENT_21,app_AMT_REQ_CREDIT_BUREAU_HOUR,app_AMT_REQ_CREDIT_BUREAU_DAY,app_AMT_REQ_CREDIT_BUREAU_WEEK,app_AMT_REQ_CREDIT_BUREAU_MON,app_AMT_REQ_CREDIT_BUREAU_QRT,app_AMT_REQ_CREDIT_BUREAU_YEAR,app_CREDIT_BY_INCOME,app_ANNUITY_BY_INCOME,app_GOODS_PRICE_BY_INCOME,app_PERCENT_WORKED,app_CNT_ADULTS,app_NUM_EXT_SOURCES,app_NUM_DOCUMENTS,app_ISNULL_BURO_ENQUIRIES,app_ISNULL_SOCIAL_CIRCLE,app_NAME_CONTRACT_TYPE_Revolving loans,app_CODE_GENDER_M,app_CODE_GENDER_XNA,app_FLAG_OWN_CAR_Y,app_FLAG_OWN_REALTY_Y,app_NAME_TYPE_SUITE_Family,app_NAME_TYPE_SUITE_Group of people,app_NAME_TYPE_SUITE_Other_A,app_NAME_TYPE_SUITE_Other_B,"app_NAME_TYPE_SUITE_Spouse, partner",app_NAME_TYPE_SUITE_Unaccompanied,app_NAME_INCOME_TYPE_Commercial associate,app_NAME_INCOME_TYPE_Maternity leave,app_NAME_INCOME_TYPE_Pensioner,app_NAME_INCOME_TYPE_State servant,app_NAME_INCOME_TYPE_Student,app_NAME_INCOME_TYPE_Unemployed,app_NAME_INCOME_TYPE_Working,app_NAME_EDUCATION_TYPE_Higher education,app_NAME_EDUCATION_TYPE_Incomplete higher,app_NAME_EDUCATION_TYPE_Lower secondary,app_NAME_EDUCATION_TYPE_Secondary / secondary special,app_NAME_FAMILY_STATUS_Married,app_NAME_FAMILY_STATUS_Separated,app_NAME_FAMILY_STATUS_Single / not married,app_NAME_FAMILY_STATUS_Unknown,app_NAME_FAMILY_STATUS_Widow,app_NAME_HOUSING_TYPE_House / apartment,app_NAME_HOUSING_TYPE_Municipal apartment,app_NAME_HOUSING_TYPE_Office apartment,app_NAME_HOUSING_TYPE_Rented apartment,app_NAME_HOUSING_TYPE_With parents,app_OCCUPATION_TYPE_Cleaning staff,app_OCCUPATION_TYPE_Cooking staff,app_OCCUPATION_TYPE_Core staff,app_OCCUPATION_TYPE_Drivers,app_OCCUPATION_TYPE_HR staff,app_OCCUPATION_TYPE_High skill tech staff,app_OCCUPATION_TYPE_IT staff,app_OCCUPATION_TYPE_Laborers,app_OCCUPATION_TYPE_Low-skill Laborers,app_OCCUPATION_TYPE_Managers,app_OCCUPATION_TYPE_Medicine staff,app_OCCUPATION_TYPE_Private service staff,app_OCCUPATION_TYPE_Realty agents,app_OCCUPATION_TYPE_Sales staff,app_OCCUPATION_TYPE_Secretaries,app_OCCUPATION_TYPE_Security staff,app_OCCUPATION_TYPE_Waiters/barmen staff,app_WEEKDAY_APPR_PROCESS_START_MONDAY,app_WEEKDAY_APPR_PROCESS_START_SATURDAY,app_WEEKDAY_APPR_PROCESS_START_SUNDAY,app_WEEKDAY_APPR_PROCESS_START_THURSDAY,app_WEEKDAY_APPR_PROCESS_START_TUESDAY,app_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,app_ORGANIZATION_TYPE_Agriculture,app_ORGANIZATION_TYPE_Bank,app_ORGANIZATION_TYPE_Business Entity Type 1,app_ORGANIZATION_TYPE_Business Entity Type 2,app_ORGANIZATION_TYPE_Business Entity Type 3,app

In [21]:
# check data
appl.shape

(356255, 211)

## 4.2. PREVIOUS APPLICATION DATA

In [25]:
# import data
prev = pd.read_csv("../data/raw/previous_application.csv")
print("Dimensions:", prev.shape)

Dimensions: (1670214, 37)


In [26]:
# check data
prev.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# count missings
nas = count_missings(prev)
nas

,Total,Percent
RATE_INTEREST_PRIVILEGED,1664263,99.643698
RATE_INTEREST_PRIMARY,1664263,99.643698
RATE_DOWN_PAYMENT,895844,53.636480
AMT_DOWN_PAYMENT,895844,53.636480
NAME_TYPE_SUITE,820405,49.119754
DAYS_TERMINATION,673065,40.298129
NFLAG_INSURED_ON_APPROVAL,673065,40.298129
DAYS_FIRST_DRAWING,673065,40.298129
DAYS_FIRST_DUE,673065,40.298129
DAYS_LAST_DUE_1ST_VERSION,673065,40.298129


In [28]:
# print factor levels
facs =  [f for f in prev.columns if prev[f].dtype == "object"]
for var in facs:
    print("----------")
    print(prev[var].value_counts())
    print(prev[var].isnull().sum(), " null values")
    print("")

----------
Cash loans         747553
Consumer loans     729151
Revolving loans    193164
XNA                   346
Name: NAME_CONTRACT_TYPE, dtype: int64
0  null values

----------
TUESDAY      255118
WEDNESDAY    255010
MONDAY       253557
FRIDAY       252048
THURSDAY     249099
SATURDAY     240631
SUNDAY       164751
Name: WEEKDAY_APPR_PROCESS_START, dtype: int64
0  null values

----------
Y    1661739
N       8475
Name: FLAG_LAST_APPL_PER_CONTRACT, dtype: int64
0  null values

----------
XAP                                 922661
XNA                                 677918
Repairs                              23765
Other                                15608
Urgent needs                          8412
Buying a used car                     2888
Building a house or an annex          2693
Everyday expenses                     2416
Medicine                              2174
Payments on other loans               1931
Education                             1573
Journey                        

In [29]:
##### FEATURE ENGINEERING

# amount ratios
prev["AMT_GIVEN_RATIO_1"]  = prev["AMT_CREDIT"] / prev["AMT_APPLICATION"]
prev["AMT_GIVEN_RATIO_2"]  = prev["AMT_GOODS_PRICE"] / prev["AMT_APPLICATION"]
prev["DOWN_PAYMENT_RATIO"] = prev["AMT_DOWN_PAYMENT"] / prev["AMT_APPLICATION"]

# logarithms
log_vars = ["AMT_CREDIT", "AMT_ANNUITY", "AMT_APPLICATION", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE"]
prev = create_logs(prev, log_vars, replace = True)

# convert days
day_vars = ["DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE", "DAYS_LAST_DUE_1ST_VERSION", 
            "DAYS_LAST_DUE", "DAYS_TERMINATION", "DAYS_DECISION"]
prev = convert_days(prev, day_vars, t = 1, rounding = False, replace = True)

# number of applications 
cnt_prev = prev[["SK_ID_CURR", "SK_ID_PREV"]].groupby(["SK_ID_CURR"], as_index = False).count()
cnt_prev.columns = ["SK_ID_CURR", "CNT_PREV_APPLICATIONS"]
prev = prev.merge(cnt_prev, how = "left", on = "SK_ID_CURR")

# number of contracts
cnt_prev = prev[["SK_ID_CURR", "FLAG_LAST_APPL_PER_CONTRACT"]]
cnt_prev.columns = ["SK_ID_CURR", "CNT_PREV_CONTRACTS"]
cnt_prev = cnt_prev[cnt_prev["CNT_PREV_CONTRACTS"] == "Y"]
cnt_prev = cnt_prev[["SK_ID_CURR", "CNT_PREV_CONTRACTS"]].groupby(["SK_ID_CURR"], as_index = False).count()
prev = prev.merge(cnt_prev, how = "left", on = "SK_ID_CURR")

# number ratio
prev["APPL_PER_CONTRACT_RATIO"] = prev["CNT_PREV_APPLICATIONS"] / prev["CNT_PREV_CONTRACTS"]

# loan decision ratios
prev = compute_accept_reject_ratio(prev, lags = [1, 3, 5])

# day differences
prev["DAYS_DUE_DIFF_1"] = prev["DAYS_LAST_DUE_1ST_VERSION"] - prev["DAYS_FIRST_DUE"]
prev["DAYS_DUE_DIFF_2"] = prev["DAYS_LAST_DUE"] - prev["DAYS_FIRST_DUE"]
prev["DAYS_TERMINATION_DIFF_1"] = prev["DAYS_TERMINATION"] - prev["DAYS_FIRST_DRAWING"]
prev["DAYS_TERMINATION_DIFF_2"] = prev["DAYS_TERMINATION"] - prev["DAYS_FIRST_DUE"]
prev["DAYS_TERMINATION_DIFF_3"] = prev["DAYS_TERMINATION"] - prev["DAYS_LAST_DUE"]

# application dates
prev["DAY_APPR_PROCESS_START"] = "Working day"
prev["DAY_APPR_PROCESS_START"][(prev["WEEKDAY_APPR_PROCESS_START"] == "SATURDAY") |
                               (prev["WEEKDAY_APPR_PROCESS_START"] == "SUNDAY")] = "Weekend"



##### FEATURE REMOVAL
drops = ["NAME_CLIENT_TYPE", "SK_ID_PREV"]
prev = prev.drop(columns = drops)



##### ADDING NULL FLAGS

# all null flags
#prev = create_null_flags(prev)



##### TREAT FACTORS

# merge some levels
#prev["NAME_TYPE_SUITE"][(prev["NAME_TYPE_SUITE"] == "Other_B") | 
#                        (prev["NAME_TYPE_SUITE"] == "Other_A")] = None
#prev["NAME_CASH_LOAN_PURPOSE"][(prev["NAME_CASH_LOAN_PURPOSE"] == "XAP")] = "XNA"
#prev["CODE_REJECT_REASON"][(prev["CODE_REJECT_REASON"] == "XAP")] = "XNA"
#prev["NAME_GOODS_CATEGORY"][(prev["NAME_GOODS_CATEGORY"] == "Other")] = "XNA"
#prev["PRODUCT_COMBINATION"][(prev["PRODUCT_COMBINATION"] == "Cash X-Sell: low") |
#                            (prev["PRODUCT_COMBINATION"] == "Cash X-Sell: middle") |
#                            (prev["PRODUCT_COMBINATION"] == "Cash X-Sell: high")] = "Cash X-Sell"
#prev["PRODUCT_COMBINATION"][(prev["PRODUCT_COMBINATION"] == "Card Street: low") |
#                            (prev["PRODUCT_COMBINATION"] == "Card Street: middle") |
#                            (prev["PRODUCT_COMBINATION"] == "Card Street: high")] = "Card Street"

# encoding
prev = treat_factors(prev, method = "dummy")

In [30]:
# aggregate features
prev = aggregate_data(prev, id_var = "SK_ID_CURR", label = "prev")

# clean up
omits = ["APPROVE_RATIO_1", "APPROVE_RATIO_3", "APPROVE_RATIO_5",  
         "REJECT_RATIO_1", "REJECT_RATIO_3",  "REJECT_RATIO_5", 
         "FLAG_LAST_APPL_PER_CONTRACT_Y", "CNT_PREV_CONTRACTS", "CNT_PREV_APPLICATIONS", 
         "APPL_PER_CONTRACT_RATIO"]
for var in omits:
    del prev["prev_" + str(var) + "_std"]
    del prev["prev_" + str(var) + "_min"]
    del prev["prev_" + str(var) + "_max"]

- Preparing the dataset...
- Extracted 0 factors and 161 numerics...
- Aggregating numeric features...
- Final dimensions: (338857, 644)


In [31]:
# check data
prev.head()

,prev_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land_mean,prev_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land_std,prev_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land_min,prev_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land_max,prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans_mean,prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans_std,prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans_min,prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans_max,prev_CODE_REJECT_REASON_XAP_mean,prev_CODE_REJECT_REASON_XAP_std,prev_CODE_REJECT_REASON_XAP_min,prev_CODE_REJECT_REASON_XAP_max,prev_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_mean,prev_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_std,prev_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_min,prev_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_max,prev_NAME_CASH_LOAN_PURPOSE_Education_mean,prev_NAME_CASH_LOAN_PURPOSE_Education_std,prev_NAME_CASH_LOAN_PURPOSE_Education_min,prev_NAME_CASH_LOAN_PURPOSE_Education_max,prev_NAME_GOODS_CATEGORY_Computers_mean,prev_NAME_GOODS_CATEGORY_Computers_std,prev_NAME_GOODS_CATEGORY_Computers_min,prev_NAME_GOODS_CATEGORY_Computers_max,prev_NAME_GOODS_CATEGORY_Audio/Video_mean,prev_NAME_GOODS_CATEGORY_Audio/Video_std,prev_NAME_GOODS_CATEGORY_Audio/Video_min,prev_NAME_GOODS_CATEGORY_Audio/Video_max,prev_NAME_GOODS_CATEGORY_Consumer Electronics_mean,prev_NAME_GOODS_CATEGORY_Consumer Electronics_std,prev_NAME_GOODS_CATEGORY_Consumer Electronics_min,prev_NAME_GOODS_CATEGORY_Consumer Electronics_max,prev_APPL_PER_CONTRACT_RATIO_mean,prev_NAME_GOODS_CATEGORY_Weapon_mean,prev_NAME_GOODS_CATEGORY_Weapon_std,prev_NAME_GOODS_CATEGORY_Weapon_min,prev_NAME_GOODS_CATEGORY_Weapon_max,prev_NAME_CASH_LOAN_PURPOSE_Car repairs_mean,prev_NAME_CASH_LOAN_PURPOSE_Car repairs_std,prev_NAME_CASH_LOAN_PURPOSE_Car repairs_min,prev_NAME_CASH_LOAN_PURPOSE_Car repairs_max,prev_NAME_CASH_LOAN_PURPOSE_Buying a garage_mean,prev_NAME_CASH_LOAN_PURPOSE_Buying a garage_std,prev_NAME_CASH_LOAN_PURPOSE_Buying a garage_min,prev_NAME_CASH_LOAN_PURPOSE_Buying a garage_max,prev_NAME_GOODS_CATEGORY_Medical Supplies_mean,prev_NAME_GOODS_CATEGORY_Medical Supplies_std,prev_NAME_GOODS_CATEGORY_Medical Supplies_min,prev_NAME_GOODS_CATEGORY_Medical Supplies_max,prev_NAME_GOODS_CATEGORY_Jewelry_mean,prev_NAME_GOODS_CATEGORY_Jewelry_std,prev_NAME_GOODS_CATEGORY_Jewelry_min,prev_NAME_GOODS_CATEGORY_Jewelry_max,prev_RATE_INTEREST_PRIMARY_mean,prev_RATE_INTEREST_PRIMARY_std,prev_RATE_INTEREST_PRIMARY_min,prev_RATE_INTEREST_PRIMARY_max,prev_PRODUCT_COMBINATION_Cash X-Sell: low_mean,prev_PRODUCT_COMBINATION_Cash X-Sell: low_std,prev_PRODUCT_COMBINATION_Cash X-Sell: low_min,prev_PRODUCT_COMBINATION_Cash X-Sell: low_max,prev_DAYS_FIRST_DUE_mean,prev_DAYS_FIRST_DUE_std,prev_DAYS_FIRST_DUE_min,prev_DAYS_FIRST_DUE_max,prev_AMT_APPLICATION_mean,prev_AMT_APPLICATION_std,prev_AMT_APPLICATION_min,prev_AMT_APPLICATION_max,prev_NAME_YIELD_GROUP_low_normal_mean,prev_NAME_YIELD_GROUP_low_normal_std,prev_NAME_YIELD_GROUP_low_normal_min,prev_NAME_YIELD_GROUP_low_normal_max,prev_NAME_CONTRACT_TYPE_Revolving loans_mean,prev_NAME_CONTRACT_TYPE_Revolving loans_std,prev_NAME_CONTRACT_TYPE_Revolving loans_min,prev_NAME_CONTRACT_TYPE_Revolving loans_max,prev_NAME_SELLER_INDUSTRY_Industry_mean,prev_NAME_SELLER_INDUSTRY_Industry_std,prev_NAME_SELLER_INDUSTRY_Industry_min,prev_NAME_SELLER_INDUSTRY_Industry_max,prev_PRODUCT_COMBINATION_POS mobile without interest_mean,prev_PRODUCT_COMBINATION_POS mobile without interest_std,prev_PRODUCT_COMBINATION_POS mobile without interest_min,prev_PRODUCT_COMBINATION_POS mobile without interest_max,prev_PRODUCT_COMBINATION_POS other with interest_mean,prev_PRODUCT_COMBINATION_POS other with interest_std,prev_PRODUCT_COMBINATION_POS other with interest_min,prev_PRODUCT_COMBINATION_POS other with interest_max,prev_NAME_PORTFOLIO_POS_mean,prev_NAME_PORTFOLIO_POS_std,prev_NAME_PORTFOLIO_POS_min,prev_NAME_PORTFOLIO_POS_max,prev_NAME_PAYMENT_TYPE_XNA_mean,prev_NAME_PAYMENT_

In [32]:
# check data
prev.shape

(338857, 614)

## 4.3. BBAL DATA

In [33]:
# import data
bbal = pd.read_csv("../data/raw/bureau_balance.csv")
print("Dimensions:", bbal.shape)

Dimensions: (27299925, 3)


In [34]:
# check data
bbal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [35]:
# count missings
nas = count_missings(bbal)
nas

,Total,Percent


In [36]:
# print factor levels
facs =  [f for f in bbal.columns if bbal[f].dtype == "object"]
for var in facs:
    print("----------")
    print(bbal[var].value_counts())
    print(bbal[var].isnull().sum(), " null values")
    print("")

----------
C    13646993
0     7499507
X     5810482
1      242347
5       62406
2       23419
3        8924
4        5847
Name: STATUS, dtype: int64
0  null values



In [37]:
##### FEATURE ENGINEERING

# number of months
cnt_bbal = bbal[["SK_ID_BUREAU", "MONTHS_BALANCE"]].groupby(["SK_ID_BUREAU"], as_index = False).count()
cnt_bbal.columns = ["SK_ID_BUREAU", "CNT_BBAL_MONTHS"]
bbal = bbal.merge(cnt_bbal, how = "left", on = "SK_ID_BUREAU")



##### FEATURE REMOVAL
drops = ["MONTHS_BALANCE"]
bbal = bbal.drop(columns = drops)



##### ADDING NULL FLAGS

# all null flags
#bbal = create_null_flags(bbal)



##### TREAT FACTORS

# merge some levels

# encoding
#bfac = bbal[["STATUS"]]
#bbal = treat_factors(bbal, method = "dummy")
#bbal = pd.concat([bbal, bfac], axis = 1)

In [38]:
# aggregate features
bbal = aggregate_data(bbal, id_var = "SK_ID_BUREAU", label = "bbal", stats = ["mean", "sum"])

# clean up mode
#del bbal["bbal_STATUS_mode"]

# clean up
omits = ["CNT_BBAL_MONTHS"]
for var in omits:
    del bbal["bbal_" + str(var) + "_sum"]

- Preparing the dataset...
- Extracted 1 factors and 1 numerics...
- Aggregating numeric features...
- Aggregating factor features...
- Final dimensions: (817395, 4)


In [39]:
# check data
bbal.head()

,bbal_CNT_BBAL_MONTHS_mean,bbal_STATUS_mode,bbal_STATUS_nunique
SK_ID_BUREAU,,,
5001709,97,C,2
5001710,83,C,3
5001711,4,0,2
5001712,19,0,2
5001713,22,X,1


In [40]:
# check data
bbal.shape

(817395, 3)

## 4.4. BUREAU DATA

In [41]:
# import data
buro = pd.read_csv("../data/raw/bureau.csv")
print("Dimensions:", buro.shape)

Dimensions: (1716428, 17)


In [42]:
# check data
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [43]:
# count missings
nas = count_missings(buro)
nas

,Total,Percent
AMT_ANNUITY,1226791,71.473490
AMT_CREDIT_MAX_OVERDUE,1124488,65.513264
DAYS_ENDDATE_FACT,633653,36.916958
AMT_CREDIT_SUM_LIMIT,591780,34.477415
AMT_CREDIT_SUM_DEBT,257669,15.011932
DAYS_CREDIT_ENDDATE,105553,6.149573
AMT_CREDIT_SUM,13,0.000757


In [44]:
# print factor levels
facs =  [f for f in buro.columns if buro[f].dtype == "object"]
for var in facs:
    print("----------")
    print(buro[var].value_counts())
    print(buro[var].isnull().sum(), " null values")
    print("")

----------
Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64
0  null values

----------
currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: CREDIT_CURRENCY, dtype: int64
0  null values

----------
Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shar

In [45]:
##### FEATURE ENGINEERING

# number of buro loans 
cnt_buro = buro[["SK_ID_CURR", "SK_ID_BUREAU"]].groupby(["SK_ID_CURR"], as_index = False).count()
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_LOANS"]
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")

# amount ratios
buro["AMT_SUM_OVERDUE_RATIO_1"] = buro["AMT_CREDIT_SUM_OVERDUE"] / buro["AMT_ANNUITY"]
buro["AMT_SUM_OVERDUE_RATIO_2"] = buro["AMT_CREDIT_SUM_OVERDUE"] / buro["AMT_CREDIT_SUM"]
buro["AMT_MAX_OVERDUE_RATIO_1"] = buro["AMT_CREDIT_MAX_OVERDUE"] / buro["AMT_ANNUITY"]
buro["AMT_MAX_OVERDUE_RATIO_2"] = buro["AMT_CREDIT_MAX_OVERDUE"] / buro["AMT_CREDIT_SUM"]
buro["AMT_SUM_DEBT_RATIO_1"]    = buro["AMT_CREDIT_SUM_DEBT"] / buro["AMT_CREDIT_SUM"]
buro["AMT_SUM_DEBT_RATIO_2"]    = buro["AMT_CREDIT_SUM_DEBT"] / buro["AMT_CREDIT_SUM_LIMIT"]

# logarithms
log_vars = ["AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT", "AMT_CREDIT_SUM_OVERDUE", "AMT_ANNUITY"]
buro = create_logs(buro, log_vars, replace = True)

# convert days
day_vars = ["DAYS_CREDIT", "CREDIT_DAY_OVERDUE", "DAYS_CREDIT_ENDDATE", "DAYS_ENDDATE_FACT", "DAYS_CREDIT_UPDATE"]
buro = convert_days(buro, day_vars, t = 1, rounding = False, replace = True)

# day differences
buro["DAYS_END_DIFF_1"] = buro["DAYS_ENDDATE_FACT"]   - buro["DAYS_CREDIT_ENDDATE"]
buro["DAYS_END_DIFF_2"] = buro["DAYS_CREDIT_UPDATE"]  - buro["DAYS_CREDIT_ENDDATE"]
buro["DAYS_DURATION_1"] = buro["DAYS_CREDIT_ENDDATE"] - buro["DAYS_CREDIT"]
buro["DAYS_DURATION_2"] = buro["DAYS_ENDDATE_FACT"]   - buro["DAYS_CREDIT"]

# number of active buro loans
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_ACTIVE"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_ACTIVE"] == "Active"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_ACTIVE"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_ACTIVE"].fillna(0, inplace = True)

# number of closed buro loans
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_CLOSED"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_CLOSED"] == "Closed"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_CLOSED"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_CLOSED"].fillna(0, inplace = True)

# number of defaulted buro loans
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_BAD"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_BAD"] == "Bad debt"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_BAD"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_BAD"].fillna(0, inplace = True)



##### MERGE ADDITIONAL DATA

# merge bbal
#buro = buro.merge(bbal, how = "left", on = "SK_ID_BUREAU")



##### FEATURE REMOVAL
drops = ["SK_ID_BUREAU"]
buro = buro.drop(columns = drops)



##### ADDING NULL FLAGS

# all null flags
#buro = create_null_flags(buro)



##### TREAT FACTORS

# merge some levels

# encoding
buro = treat_factors(buro, method = "dummy")

In [46]:
# aggregate features
buro = aggregate_data(buro, id_var = "SK_ID_CURR", label = "buro")

# clean up
omits = ["CNT_BURO_LOANS", "CNT_BURO_ACTIVE", "CNT_BURO_CLOSED", "CNT_BURO_BAD"]
for var in omits:
    del buro["buro_" + str(var) + "_std"]
    del buro["buro_" + str(var) + "_min"]
    del buro["buro_" + str(var) + "_max"]

- Preparing the dataset...
- Extracted 0 factors and 65 numerics...
- Aggregating numeric features...
- Final dimensions: (305811, 260)


In [47]:
# check data
buro.head()

,buro_DAYS_END_DIFF_2_mean,buro_DAYS_END_DIFF_2_std,buro_DAYS_END_DIFF_2_min,buro_DAYS_END_DIFF_2_max,buro_CNT_BURO_BAD_mean,buro_CNT_CREDIT_PROLONG_mean,buro_CNT_CREDIT_PROLONG_std,buro_CNT_CREDIT_PROLONG_min,buro_CNT_CREDIT_PROLONG_max,buro_ISNULL_AMT_SUM_OVERDUE_RATIO_2_mean,buro_ISNULL_AMT_SUM_OVERDUE_RATIO_2_std,buro_ISNULL_AMT_SUM_OVERDUE_RATIO_2_min,buro_ISNULL_AMT_SUM_OVERDUE_RATIO_2_max,buro_CREDIT_TYPE_Mobile operator loan_mean,buro_CREDIT_TYPE_Mobile operator loan_std,buro_CREDIT_TYPE_Mobile operator loan_min,buro_CREDIT_TYPE_Mobile operator loan_max,buro_ISNULL_DAYS_CREDIT_ENDDATE_mean,buro_ISNULL_DAYS_CREDIT_ENDDATE_std,buro_ISNULL_DAYS_CREDIT_ENDDATE_min,buro_ISNULL_DAYS_CREDIT_ENDDATE_max,buro_CREDIT_ACTIVE_Closed_mean,buro_CREDIT_ACTIVE_Closed_std,buro_CREDIT_ACTIVE_Closed_min,buro_CREDIT_ACTIVE_Closed_max,buro_CREDIT_TYPE_Car loan_mean,buro_CREDIT_TYPE_Car loan_std,buro_CREDIT_TYPE_Car loan_min,buro_CREDIT_TYPE_Car loan_max,buro_CREDIT_TYPE_Interbank credit_mean,buro_CREDIT_TYPE_Interbank credit_std,buro_CREDIT_TYPE_Interbank credit_min,buro_CREDIT_TYPE_Interbank credit_max,buro_ISNULL_DAYS_END_DIFF_2_mean,buro_ISNULL_DAYS_END_DIFF_2_std,buro_ISNULL_DAYS_END_DIFF_2_min,buro_ISNULL_DAYS_END_DIFF_2_max,buro_AMT_CREDIT_SUM_mean,buro_AMT_CREDIT_SUM_std,buro_AMT_CREDIT_SUM_min,buro_AMT_CREDIT_SUM_max,buro_CREDIT_TYPE_Microloan_mean,buro_CREDIT_TYPE_Microloan_std,buro_CREDIT_TYPE_Microloan_min,buro_CREDIT_TYPE_Microloan_max,buro_ISNULL_AMT_ANNUITY_mean,buro_ISNULL_AMT_ANNUITY_std,buro_ISNULL_AMT_ANNUITY_min,buro_ISNULL_AMT_ANNUITY_max,buro_AMT_CREDIT_SUM_DEBT_mean,buro_AMT_CREDIT_SUM_DEBT_std,buro_AMT_CREDIT_SUM_DEBT_min,buro_AMT_CREDIT_SUM_DEBT_max,buro_CREDIT_TYPE_Real estate loan_mean,buro_CREDIT_TYPE_Real estate loan_std,buro_CREDIT_TYPE_Real estate loan_min,buro_CREDIT_TYPE_Real estate loan_max,buro_ISNULL_AMT_MAX_OVERDUE_RATIO_2_mean,buro_ISNULL_AMT_MAX_OVERDUE_RATIO_2_std,buro_ISNULL_AMT_MAX_OVERDUE_RATIO_2_min,buro_ISNULL_AMT_MAX_OVERDUE_RATIO_2_max,buro_ISNULL_DAYS_DURATION_1_mean,buro_ISNULL_DAYS_DURATION_1_std,buro_ISNULL_DAYS_DURATION_1_min,buro_ISNULL_DAYS_DURATION_1_max,buro_AMT_SUM_OVERDUE_RATIO_1_mean,buro_AMT_SUM_OVERDUE_RATIO_1_std,buro_AMT_SUM_OVERDUE_RATIO_1_min,buro_AMT_SUM_OVERDUE_RATIO_1_max,buro_CREDIT_TYPE_Consumer credit_mean,buro_CREDIT_TYPE_Consumer credit_std,buro_CREDIT_TYPE_Consumer credit_min,buro_CREDIT_TYPE_Consumer credit_max,buro_CREDIT_TYPE_Credit card_mean,buro_CREDIT_TYPE_Credit card_std,buro_CREDIT_TYPE_Credit card_min,buro_CREDIT_TYPE_Credit card_max,buro_AMT_SUM_DEBT_RATIO_1_mean,buro_AMT_SUM_DEBT_RATIO_1_std,buro_AMT_SUM_DEBT_RATIO_1_min,buro_AMT_SUM_DEBT_RATIO_1_max,buro_ISNULL_AMT_CREDIT_MAX_OVERDUE_mean,buro_ISNULL_AMT_CREDIT_MAX_OVERDUE_std,buro_ISNULL_AMT_CREDIT_MAX_OVERDUE_min,buro_ISNULL_AMT_CREDIT_MAX_OVERDUE_max,buro_CREDIT_ACTIVE_Sold_mean,buro_CREDIT_ACTIVE_Sold_std,buro_CREDIT_ACTIVE_Sold_min,buro_CREDIT_ACTIVE_Sold_max,buro_AMT_SUM_OVERDUE_RATIO_2_mean,buro_AMT_SUM_OVERDUE_RATIO_2_std,buro_AMT_SUM_OVERDUE_RATIO_2_min,buro_AMT_SUM_OVERDUE_RATIO_2_max,buro_CREDIT_TYPE_Mortgage_mean,buro_CREDIT_TYPE_Mortgage_std,buro_CREDIT_TYPE_Mortgage_min,buro_CREDIT_TYPE_Mortgage_max,buro_CREDIT_TYPE_Loan for purchase of shares (margin lending)_mean,buro_CREDIT_TYPE_Loan for purchase of shares (margin lending)_std,buro_CREDIT_TYPE_Loan for purchase of shares (margin lending)_min,buro_CREDIT_TYPE_Loan for purchase of shares (margin lending)_max,buro_ISNULL_DAYS_CREDIT_UPDATE_mean,buro_ISNULL_DAYS_CREDIT_UPDATE_std,buro_ISNULL_DAYS_CREDIT_UPDATE_min,buro_ISNULL_DAYS_CREDIT_UPDATE_max,buro_CREDIT_TYPE_Loan for working capital replenishment_mean,buro_CREDIT_TYPE_Loan for working capital replenishment_std,buro_CREDIT_TYPE_Loan for working capital replenishment_min,buro_CREDIT_TYPE_Loan for working capital replenishment_max,buro_DAYS_CREDIT_ENDDATE_mean,buro_DAYS_CREDIT_ENDDATE_std,buro_DAYS_CREDIT_ENDDATE_min,buro_DAYS_CREDIT_ENDDATE_max,buro_ISNULL_AMT_SUM_OVERDUE_RATIO_1_mean,bur

In [48]:
# check data
buro.shape

(305811, 248)

# 5. EXPORT DATA

In [ ]:
# merging
appl = appl.merge(prev.reset_index(), how = "left", on = "SK_ID_CURR")
appl = appl.merge(buro.reset_index(), how = "left", on = "SK_ID_CURR")
del prev, buro

In [50]:
##### CROSS-TABLE FEATURE ENGINEERING

# credit ratios
appl["mix_AMT_PREV_ANNUITY_RATIO"]     = appl["app_AMT_ANNUITY"] / appl["prev_AMT_ANNUITY_mean"]
appl["mix_AMT_PREV_CREDIT_RATIO"]      = appl["app_AMT_CREDIT"] / appl["prev_AMT_CREDIT_mean"]
appl["mix_AMT_PREV_GOODS_PRICE_RATIO"] = appl["app_AMT_GOODS_PRICE"] / appl["prev_AMT_GOODS_PRICE_mean"]
appl["mix_AMT_BURO_ANNUITY_RATIO"]     = appl["app_AMT_ANNUITY"] / appl["buro_AMT_ANNUITY_mean"]
appl["mix_AMT_BURO_CREDIT_RATIO"]      = appl["app_AMT_CREDIT"] / appl["buro_AMT_CREDIT_SUM_mean"]

In [23]:
# partitioning
train = appl[appl["SK_ID_CURR"].isin(y["SK_ID_CURR"]) == True]
test  = appl[appl["SK_ID_CURR"].isin(y["SK_ID_CURR"]) == False]
del appl

In [24]:
# check dimensions
print(train.shape)
print(test.shape)

(307511, 211)
(48744, 211)


In [25]:
# export CSV
train.to_csv("../data/prepared/train_app.csv", index = False, float_format = "%.8f")
test.to_csv("../data/prepared/test_app.csv",   index = False, float_format = "%.8f")
y.to_csv("../data/prepared/y_app.csv",         index = False, float_format = "%.8f")